# Circuit Generation

Construct a memory experiment circuit and inspect the output.


### Error-rate parameters used in `get_qldpc_mem_circuit`

The call `get_qldpc_mem_circuit(code, p, p, p, p, num_rounds, ...)` uses the same scalar `p` for all noise channels:

- `idle_error`: depolarizing error rate on idle qubits
- `sqgate_error`: single-qubit gate depolarizing error rate
- `tqgate_error`: two-qubit gate depolarizing error rate
- `spam_error`: state-preparation and measurement error rate

You can pass different values (or tuples) to model asymmetric noise if desired.

If you pass a **tuple** instead of a single float, it specifies a *Pauli channel*:

- `idle_error` and `sqgate_error`: `(px, py, pz)` for `PAULI_CHANNEL_1` (one-qubit noise)
- `tqgate_error`: a 15-tuple `(pix, piy, piz, pxi, ..., pzz)` for `PAULI_CHANNEL_2` (two-qubit noise)

These map directly to Stim's `PAULI_CHANNEL_1` and `PAULI_CHANNEL_2` instructions, letting you model asymmetric noise instead of uniform depolarizing noise.


In [1]:
import numpy as np
import stim

from quits.circuit import get_qldpc_mem_circuit, check_overlapping_CX
from quits.decoder import (
    sliding_window_bposd_circuit_mem,
    sliding_window_bposd_phenom_mem,
    sliding_window_bplsd_phenom_mem,
)
from quits.qldpc_code import BpcCode, HgpCode, QlpCode
from quits.simulation import get_stim_mem_result


In [2]:
# Common parameters
seed = 1
p = 1e-3
num_rounds = 10
num_trials = 100
W, F = 5, 3
max_iter, osd_order = 10, 1


In [3]:
# Build a small HGP code
h = np.loadtxt(
    "../parity_check_matrices/n=12_dv=3_dc=4_dist=6.txt",
    dtype=int,
)
code = HgpCode(h, h)
code.build_circuit(seed=22)
# For this particular parity check matrix, seed=22 is the one that gives entangling depth 8 (other seeds may give entangling depth 12).
report = code.verify_css_logicals()
print("verify_css_logicals", report)


verify_css_logicals {'css_condition': True, 'lz_commutes_with_X': True, 'lx_commutes_with_Z': True, 'rank_hz': 108, 'rank_hx': 108, 'rank_lz': 9, 'rank_lx': 9, 'k_expected': 9, 'lz_independent_mod_Z_stabilizers': True, 'lx_independent_mod_X_stabilizers': True, 'rank_hz_plus_lz': 117, 'rank_hx_plus_lx': 117, 'dim_ker_hz': 117, 'dim_ker_hx': 117, 'hx_plus_lx_spans_ker_hz': True, 'hz_plus_lz_spans_ker_hx': True, 'pairing_rank': 9, 'pairing_is_identity': True, 'ok': True}


In [5]:
# Generate memory experiment circuit
circuit = stim.Circuit(
    get_qldpc_mem_circuit(code, p, p, p, p, num_rounds, basis="Z")
)
check_overlapping_CX(circuit, verbose=True);
# print(circuit)


No overlapping CX gates found.
